In [ ]:
import psycopg2

API_KEY = '2CDC7D3628A900A6E59E9BC19A8E0B62'

try:
    connection = psycopg2.connect(
        host="26.84.71.27",       
        database="DataScienceProj",
        user="postgres",     
        password="SteamDB",    
        port="5432",
        options="-c client_encoding=UTF8"  
    )

    cursor = connection.cursor()

    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print("Versão do PostgreSQL:", version)

except Exception as error:
    print("Erro ao conectar ao banco de dados:", str(error).encode('utf-8', errors='replace').decode())


In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

query = """
SELECT DISTINCT id, 
       (SELECT description 
        FROM steam.games_categories_steam cat 
        WHERE cat.id = a.id LIMIT 1) AS description, 
       qtdejogos 
FROM (
    SELECT DISTINCT id, 
           gcs.description, 
           COUNT(appid) OVER (PARTITION BY gcs.id) AS qtdejogos
    FROM steam.games_categories_steam gcs
) AS a 
ORDER BY qtdejogos DESC
"""
data = pd.read_sql_query(query, connection)

connection.close()
plt.figure(figsize=(12, 8))
sns.barplot(data=data, x='qtdejogos', y='description', palette='viridis')
plt.title("Quantidade de Jogos por Categoria", fontsize=16)
plt.xlabel("Quantidade de Jogos", fontsize=14)
plt.ylabel("Categoria", fontsize=14)
plt.xlim(0, data['qtdejogos'].max() + 5) 

plt.tight_layout()

plt.show()

query2 = """ SELECT * FROM """

In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


if connection:
    # Definir a consulta SQL
    query = """
    SELECT GI.NAME, 
           G.last_week_rank, 
           (cast(G.peak_in_game as float) / 1000) as peak_in_game
    FROM STEAM.GAMES G
    JOIN STEAM.GAME_INFO GI ON G.APPID = GI.steam_appid
    ORDER BY RANK
    """

    try:
        # Executar a consulta e carregar os dados em um DataFrame
        data = pd.read_sql_query(query, connection)
        connection.close()
        print("Conexão encerrada.")

        # Verificar o DataFrame
        if data.empty:
            print("O DataFrame está vazio. Verifique sua consulta SQL.")
        else:
            print(data.head())  # Exibir as primeiras linhas

            # Certificar-se de que as colunas têm os nomes corretos
            data.columns = [col.strip().upper() for col in data.columns]

            # Plotar o gráfico
            plt.figure(figsize=(12, 18))
            sns.barplot(data=data, x='PEAK_IN_GAME', y='NAME', palette='viridis')

            # Adicionar rótulos com os valores reais
            for index, value in enumerate(data['PEAK_IN_GAME']):
                plt.text(value, index, f"{value:.1f}", va='center', fontsize=10)

            # Configurar título e rótulos
            plt.title("Jogos com Maior Número de Jogadores em Pico (a cada mil jogadores)", fontsize=16)
            plt.xlabel("Pico de Jogadores (em milhares)", fontsize=12)
            plt.ylabel("Nome do Jogo", fontsize=12)

            # Ajustar limites do eixo X automaticamente
            max_peak = data['PEAK_IN_GAME'].max()
            plt.xlim(0, max_peak * 1.1)  # Adicionar 10% de margem

            plt.tight_layout()

            # Exibir o gráfico
            plt.show()
    except Exception as e:
        print(f"Erro ao executar a consulta ou gerar o gráfico: {e}")
